In [1]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from sql_queries import summary_table_insert
from psycopg2.extras import execute_batch

In [29]:
class Scraper:
    def __init__(self):
        self.driver = None
        self.conn = None
        self.cur = None
        self.test_attribute = 5
    
    def open_driver(self):
        self.driver = open_driver()
    
    def close_driver(self):
        if self.driver:
            close_driver(self.driver)

In [33]:
class SummaryScraper(Scraper):
    def __init__(self):
        super().__init__()
    
    def db_connect(self):
        self.conn, self.cur = db_connect()
    
    def close_db_connect(self):
        if self.conn and self.
        close_db_connect(self.conn, self.cur)

In [34]:
scraper = SummaryScraper()

In [32]:
scraper.open_driver()
scraper.close_driver()

In [ ]:
scraper.db_connect()